In [1]:
import os

In [2]:
%pwd

'c:\\Users\\PASCAL\\Car_Price_Predictions\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PASCAL\\Car_Price_Predictions'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    

In [6]:
from src.CarPrice.constants import *
from src.CarPrice.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir=config.root_dir,
        source_URL=config.source_URL,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir
        
    )

        return data_ingestion_config
 

In [8]:
import os
import urllib.request as request
import zipfile
from CarPrice.logger import logging
from CarPrice.utils.common import get_size
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from CarPrice.logger import logging
from sklearn.model_selection import train_test_split
import pandas as pd
# import requests
import urllib
import sys
import zipfile

In [9]:


class DataIngestion:
    def __init__(self, data_ingestion_config):
        self.config = data_ingestion_config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded with the following info:\n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def ingest_data(self):
        logging.info("Ingestion of the data is initiated")

        raw_data_path = os.path.join(self.config.unzip_dir, "raw_data.csv")
        train_data_path = os.path.join(self.config.unzip_dir, "train_data.csv")
        test_data_path = os.path.join(self.config.unzip_dir, "test_data.csv")

        data_files = os.listdir(self.config.unzip_dir)
        csv_files = [file_name for file_name in data_files if file_name.endswith(".csv")]

        if len(csv_files) == 0:
            raise ValueError("No CSV files found in the unzip directory.")

        df = None
        for file_name in csv_files:
            file_path = os.path.join(self.config.unzip_dir, file_name)
            if df is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.concat([df, pd.read_csv(file_path)], ignore_index=True)

        # Save the raw data
        df.to_csv(raw_data_path, index=False, header=True)

        # Train test split
        train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

        # Save the train and test data
        train_set.to_csv(train_data_path, index=False, header=True)
        test_set.to_csv(test_data_path, index=False, header=True)

        logging.info("Ingestion of the data is completed")

        return train_data_path, test_data_path



In [11]:


class DataIngestion:
    def __init__(self, data_ingestion_config):
        self.config = data_ingestion_config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded with the following info:\n{headers}")
        else:
            print(f"File already exists of size: {os.path.getsize(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            print("ZIP file extracted successfully.")
        except zipfile.BadZipFile:
            print("BadZipFile: File is not a valid ZIP file.")
        except Exception as e:
            print(f"An error occurred while extracting the ZIP file: {str(e)}")

    def ingest_data(self):
        logging.info("Ingestion of the data is initiated")

        raw_data_path = os.path.join(self.config.unzip_dir, "raw_data.csv")
        train_data_path = os.path.join(self.config.unzip_dir, "train_data.csv")
        test_data_path = os.path.join(self.config.unzip_dir, "test_data.csv")

        data_files = os.listdir(self.config.unzip_dir)
        csv_files = [file_name for file_name in data_files if file_name.endswith(".csv")]

        if len(csv_files) == 0:
            print("No CSV files found in the unzip directory.")
            return None, None  # Return None values if no CSV files are found

        df = None
        for file_name in csv_files:
            file_path = os.path.join(self.config.unzip_dir, file_name)
            if df is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.concat([df, pd.read_csv(file_path)], ignore_index=True)

        # Save the raw data
        df.to_csv(raw_data_path, index=False, header=True)

        # Train test split
        train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

        # Save the train and test data
        train_set.to_csv(train_data_path, index=False, header=True)
        test_set.to_csv(test_data_path, index=False, header=True)

        logging.info("Ingestion of the data is completed")

        return train_data_path, test_data_path


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    train_data_path, test_data_path = data_ingestion.ingest_data()
except Exception as e:
    raise e

[2023-09-25 17:48:22,295: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-25 17:48:22,297: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-25 17:48:22,299: INFO: common: created directory at: artifacts]
[2023-09-25 17:48:22,301: INFO: common: created directory at: artifacts/data_ingestion]
File already exists of size: 121442
BadZipFile: File is not a valid ZIP file.
[2023-09-25 17:48:22,303: INFO: 1428563508: Ingestion of the data is initiated]
No CSV files found in the unzip directory.
